In [1]:
!date
!python --version

Thu May 30 04:39:14 AM UTC 2024
Python 3.10.12


# テキストをトークン出現回数でベクトル化する例（Spacy版）
基本的には「形態素解析して単語に分割し、その回数をカウントしたうえでベクトル化する」という手順を取る。形態素解析には様々なツールがあるが、ここでは ``spacy.load("ja_ginza")`` を用いた例を示す。単語分割した文字列を作成したら、後は[CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)を使うのが楽だ。

In [2]:
!pip install -U spacy ja_ginza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.1 MB/s eta 0:00:00


## spacyで単語分割
- 単に分割するだけではなく、``token.lemma_`` により基本形に変換している。元のままが良ければ ``token.text`` にしよう。
- タスクによっては不要な単語や品詞もあるだろう。その場合には不要なものを除外しよう。
- タスクによっては集約（例えば数字を全て`<数字>`という単語に集約する。感情語を全て`<感情>`に集約する）を検討すると良いだろう。

In [3]:
import spacy

# テキスト例
texts = ['特になし',
        '正直わかりずらい。むだに間があるし。',
        '例題を取り入れて理解しやすくしてほしい。']

# 解析器を用意
nlp = spacy.load("ja_ginza")

# 解析例
doc = nlp("正直わかりずらい。")
for token in doc:
    print(f"{token.i=}, {token.text=}, {token.lemma_=}")

token.i=0, token.text='正直', token.lemma_='正直'
token.i=1, token.text='わかり', token.lemma_='わかる'
token.i=2, token.text='ずらい', token.lemma_='ずらい'
token.i=3, token.text='。', token.lemma_='。'


In [4]:
def text2tokens(nlp:spacy.language.Language, text:str, sep=' '):
    """テキストを単語に分割した文字列に変換。
    args:
      nlp: spacy.load()で用意した解析器。
      text: テキスト。
      sep: セパレータ。単語と単語の間を埋める記号。

    >>> nlp = spacy.load("ja_ginza")
    >>> result = text2tokens(nlp, "これはテストです")
    >>> result
    'これ は テスト です'
    """
    doc = nlp(text)
    tokens = []
    for token in doc:
        tokens.append(token.lemma_)
    result = sep.join(tokens)
    return result

# 実行例
tokens = []
for text in texts:
    tokens.append(text2tokens(nlp, text))

tokens

['特に なし', '正直 わかる ずらい 。 むだ だ 間 が ある し 。', '例題 を 取り入れる て 理解 する やすい する て ほしい 。']

## CountVectorizerでベクトル化
デフォルトでは1-gramモデル（各単語の出現回数に基づいた特徴）によりベクトル化する。引数指定により以下のような設定も可能。[詳細はドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)参照。
- `ngram_range`
    - 2-gram, 3-gram,,,といった「連続した語＝フレーズ」に基づいたベクトル化を行う。
- `stop_words`
    - 無視したい単語（ストップワード）を指定することができる。標準で用意されているリストを利用することも可能。
- `analyzer`
    - デフォルトでは単語を特徴として捉えるが、この単語とは「スペースで区切られたもの」として解釈される。
    - 'char' を指定すると「文字」を特徴として捉えるようになる。

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# 1-gramで特徴ベクトル作成
vectorizer = CountVectorizer() # デフォルトでは単語出現回数でベクトル化
X = vectorizer.fit_transform(tokens) # ベクトル構築
features = vectorizer.get_feature_names_out() # ベクトル構築した際の単語一覧
print(f"{features=}")
print(X.toarray())

features=array(['ある', 'する', 'ずらい', 'なし', 'ほしい', 'むだ', 'やすい', 'わかる', '例題', '取り入れる',
       '正直', '特に', '理解'], dtype=object)
[[0 0 0 1 0 0 0 0 0 0 0 1 0]
 [1 0 1 0 0 1 0 1 0 0 1 0 0]
 [0 2 0 0 1 0 1 0 1 1 0 0 1]]


In [6]:
# 2-gramで特徴ベクトル作成
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(tokens)
features = vectorizer.get_feature_names_out()
print(f"{features=}")
print(X.toarray())

features=array(['する ほしい', 'する やすい', 'ずらい むだ', 'むだ ある', 'やすい する', 'わかる ずらい',
       '例題 取り入れる', '取り入れる 理解', '正直 わかる', '特に なし', '理解 する'], dtype=object)
[[0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 1 0 1 0 0 1 0 0]
 [1 1 0 0 1 0 1 1 0 0 1]]
